# *Frequently Asked Questions*

This document lists known issues and tips and tricks for working with the `pypovray` project.

# Table of Contents

* [**I Get The TypeError: Can't convert 'bytes' object to str implicitly**](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/FAQ.ipynb#I-Get-The-TypeError:-Can't-convert-'bytes'-object-to-str-implicitly)

* [**I Get The 'movie.mp4' already exists. Not overwriting - exiting Error**](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/FAQ.ipynb#I-Get-The-'movie.mp4'-already-exists.-Not-overwriting---exiting-Error)

* [**Running Povray through SSH Doesn't Work: ImportError: No module named 'vapory'**](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/FAQ.ipynb#Running-Povray-through-SSH-Doesn't-Work:-ImportError:-No-module-named-'vapory')

* [**I want to switch between configuration files; How do I do this?**](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/FAQ.ipynb#I-want-to-switch-between-configuration-files;-How-do-I-do-this?)

* [**When I run my program, nothing happens!**](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/FAQ.ipynb#When-I-run-my-program,-nothing-happens!)

* [**I Found a Bug! Now What?**](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/FAQ.ipynb#I-Found-a-Bug!-Now-What?)

# I Get The TypeError: Can't convert 'bytes' object to str implicitly

This is the most-seen error when working with the `Vapory` library and will look something like this:

    Traceback (most recent call last):
      File "test.py", line 44, in <module>
        povray.make_frame(0, scene, time=True)
      File "povray/povray.py", line 39, in make_frame
        quality=SETTINGS.Quality, remove_temp=False)
      File "/lib/python3.5/site-packages/vapory/vapory.py", line 102, in render
        quality, antialiasing, remove_temp)
      File "python/povray/lib/python3.5/site-packages/vapory/io.py", line 114, in render_povstring
        raise IOError("POVRay rendering failed with the following error: "+err)
    TypeError: Can't convert 'bytes' object to str implicitly

While normally Python errors are pretty descriptive (it will show what is wrong and where it occurs, you will get this error every time there is *something* wrong with one of your Povray objects.

As you can see in the code below there is an *invalid* `color` declaration, this *should* be done with:

    sphere = Sphere([2, 2, 0], 2, Pigment('color', 'Gold'))

In [1]:
from povray import povray
from vapory import Scene, Sphere

def scene(step):
    sphere = Sphere([2, 2, 0], 2, 'color', 'Gold')
    return Scene(povray.default_camera,
                 objects=[povray.default_light, sphere],
                 included=['colors.inc'])

povray.make_frame(0, scene, time=True)

TypeError: Can't convert 'bytes' object to str implicitly

Note the `included=[]` added to the `Scene` object, see the [Povray Objects and Styling](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/povray_objects.ipynb) manual for further details.

## Debugging

These errors are not always easy to spot since we get no feedback *at all* about where something is wrong. Therefore the easiest method of *debugging* this error is the following.

For example in assignment 2 we created a scene consisting of about 9 objects, a camera and a lightsource. Our `Scene` object might look something like this:

    Scene(povray.default_camera, objects=[box1, box2, box3, box4,
                                          cone1, cone2, cone3, cone4,
                                          povray.default_lightsource, sphere])

If we now get the error and want to find out *which* object causes it, we could modify **only** the `Scene` object as follows:

    Scene(povray.default_camera, objects=[#box1, box2, box3, box4,
                                          #cone1, cone2, cone3, cone4,
                                          povray.default_lightsource, sphere])

We now *commented out* all the `Boxes` and `Cones` (they won't be included in the image) and after running our script we see that there is no issue anymore. This means that the error is caused by either one or more of the `Boxes` or `Cones`. So, the next step would be to change our `Scene` to:

    Scene(povray.default_camera, objects=[box1, box2, box3, box4,
                                          #cone1, cone2, cone3, cone4,
                                          povray.default_lightsource, sphere])

where we now added all the `Boxes` and kept the `Cones` hidden. If we still get the error, we now know the problem lies with the `Boxes`. Now we can refer to the [Povray Objects and Styling](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/povray_objects.ipynb) manual to see if we made an obvious mistake somewhere. If we did not get the error however it is clear that it is one of the `Cones` causing it. You can then proceed to exclude each `Cone` separately to find the mistake.

# I Get The 'movie.mp4' already exists. Not overwriting - exiting Error

The `povray` packages uses the `ffmpeg` software to create mp4 video files. This software does not overwrite existing files, so getting this error message means that you already have an mp4 file with the name as configured in the configuration (`.ini`) file.

To solve this, rename the existing file to something different or - before running your simulation - change the `OutputPrefix` parameter in the configuration file to give the new output file a different name.


# Running Povray through SSH Doesn't Work: `ImportError: No module named 'vapory'`

When I login through SSH on the `bioinf.nl` netwerk, I cannot run my `povray` code, getting the `ImportError: No module named 'vapory'` error.

This is intended behaviour and easy to solve. When you login through SSH you 'land' on one of our servers (mordor) which should *only* be used as a login-server. This means that you should connect to another system on the network to do the actual work. So, once you are logged in on mordor you perform another SSH command to reach a different system;

    ssh bin255

which lands you on a workstation in the H1.12 room and here you can perform your assignments. While it is perfectly fine to use one of our workstations to run your `povray` code to create a movie, this does create a heavy load on the system and should only be done when no one else is using it (you can check with the `who` command or on our [`status`](https://bioinf.nl/~piet/status) webpage).

## Rendering Movies through SSH

To render a movie or a very comlex single image, it is best to SSH to one of our bigger servers that can more easily sustain such heavy loads and have more processing power too. Select either `idefix` (24 cores, 96GB RAM) or `assemblix` (80 cores, 512GB RAM) for such tasks using one of these commands:

    ssh idefix
    ssh assemblix

# I want to switch between configuration files; How do I do this?

The `pypovray` project includes two configuration files; the `default.ini` and the `prototype.ini`. The `default.ini` file is used every time you create an image or a simulation but as you will find out the rendering can take a lot of time. This is mostly because the `default.ini` file creates a high-resolution image with high quality settings. So during development it might be better to use a lower resolution output with lower quality to get your results more quickly. The `prototype.ini` specifies exactly that, so it is recommended to use this file during development. The following code shows how to do this:

In [4]:
from povray import povray, load_config
from vapory import Scene, Sphere, Pigment

def scene(step):
    sphere = Sphere([step * 0.2, 2, 0], 2, Pigment('color', 'Gold'))
    return Scene(povray.default_camera,
                 objects=[povray.default_light, sphere],
                 included=['colors.inc'])

if __name__ == '__main__':
    # Load a different settings file
    povray.SETTINGS = load_config('prototype.ini')
    
    # Show that the settings have changed
    print(povray.SETTINGS.Duration, 's x ', povray.SETTINGS.RenderFPS, 'FPS = ',
          povray.SETTINGS.Duration * povray.SETTINGS.RenderFPS, ' total frames.', sep='')

    # Render a movie
    povray.render_scene_to_mp4(scene, time=False)

6.0s x 15.0FPS = 90.0 total frames.


As you can see these settings are now used from the `prototype.ini` file:

    Duration = 6
    RenderFPS = 15

where the `default.ini` file has these settings resulting in 180 frames in total:

    Duration = 6
    RenderFPS = 30



# When I run my program, nothing happens!

If you have a complete program (such as the `template.py` file included in the project) that also has a `main` function and it doesn't do anything when running it from the commandline, there might be an issue with the imports.

The code example shown above in the [*I want to switch between configuration files; How do I do this?*](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/pypovray/raw/master/manual/FAQ.ipynb#I-want-to-switch-between-configuration-files;-How-do-I-do-this?) question shows the standard

    if __name__ == "__main__":
        ...
        
statement included in all our code. This *should* always work, but there is one situation in which it doesn't. Check your code to see if you have the following include statement:

    from vapory import *

If so, this is the reason for your program doing absolutely *nothing*! This is because the contents of `__name__` has changed to the string `vapory`. This is the only package we know of with this non-standard behaviour, so the following workaround can be applied in this case. Either change the `if`-statement to:

    if __name__ == "vapory":
        ...

, or change your import to include all the objects you need (you can use PyCharm to manage your imports):

    from vapory import Scene, Camera, LightSource, Sphere, Cylinder, Cone, Texture, Pigment, etc...



# I Found a Bug! Now What?

Okay, so you have found something amiss with part of the software. Depending on where this bug lies you can contact the author of the software. In case you have found a Python error (most unlikely) you can read the respective [document](https://docs.python.org/3.4/bugs.html), but if it is a bug with the `povray` library, please contact its developer by reporting it in the [*issue tracker*](https://bitbucket.org/mkempenaar/pypovray/issues). Reported issues will be fixed or a workaround will be placed here in this FAQ, thank you for your support!